**=====================================================**  
# Grain Storage Analysis

Noah MacGillivray | BIT-446 Assignment(T8) | Summer 2025 

Random Forest 

**Environment:** Ubuntu, VS Code, Python Version: 3.10.12  

**=====================================================**

## Package Imports 



In [2]:
# Data manipulation and analysis
import pandas as pd
import numpy as np

# File handling
import os
import openpyxl

# Machine learning
import sklearn

# Data visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Display options
pd.set_option('display.max_columns', None)

print("Environment setup complete!")

Environment setup complete!
